In [1]:
#--------------------------------Liberias---------------------------------
import os
import warnings
warnings.filterwarnings('ignore')

# PySpark
os.environ["HADOOP_HOME"] = "C:\\hadoop" # Configuración de Spark/Hadoop
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import col, rand
from pyspark.sql.types import DoubleType, StructType, StructField

# Keras/TensorFlow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Elephas - Deep Learning sobre Spark
from elephas.spark_model import SparkModel
from elephas.utils.rdd_utils import to_simple_rdd
#---------------------------------------------------------------------------------

In [2]:
#----------------Crear SparkSession-----------------------------------------------
spark = SparkSession.builder \
    .appName("RedNeuronal_Elephas") \
    .master("local[*]") \
    .config("spark.driver.memory", "10g") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.maxResultSize", "8g") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
#---------------------------------------------------------------------------------

In [3]:
#----------------------Cargar datos-------------------------------------------------------
DATA_PATH = "C:/Users/PC/Documents/DocumentosGustavo/Github/Maestria/BigData/nyc-taxi-spark/data/yellow/2024/yellow_tripdata_2024-01.parquet"
df = spark.read.parquet(DATA_PATH)
df.show(5) #Se utiliza show para vizualizar los datos, es como head en pandas, aqui solo vizualiso el parquet de enero

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [4]:
print(f"  Total de registros: {df.count()}")
print(f"  Columnas: {len(df.columns)}")
print("\nEsquema del dataset:")
df.printSchema()

  Total de registros: 2964624
  Columnas: 19

Esquema del dataset:
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [5]:
# Seleccionar las columnas necesarias
rdd_features = df.select(
    "trip_distance",
    "passenger_count", 
    "tpep_pickup_datetime",
    "fare_amount"
).rdd.map(lambda row: (
    row.trip_distance,
    row.passenger_count,
    row.tpep_pickup_datetime,
    row.fare_amount
))

print(f" Se creocon exito el RDD con las columnas seleccionadas.")
print(f"  Total registros: {rdd_features.count():,}")
print(f"\nEjemplo de registro:")
print(rdd_features.take(1)[0])

 Se creocon exito el RDD con las columnas seleccionadas.
  Total registros: 2,964,624

Ejemplo de registro:
(1.72, 1, datetime.datetime(2024, 1, 1, 0, 57, 55), 17.7)


In [6]:
#----------------------Feature Engineering con Escalado Estándar-----------------
def extract_and_scale_features(row):
    """   Extrae features y aplica escalado (Z-score) para usar con ReLU.
    Input: (trip_distance, passenger_count, datetime, fare_amount)
    Output: (features_list, label_list) o None si datos inválidos
    """
    trip_distance, passenger_count, datetime, fare_amount = row
    
    # ========== FILTROS DE VALIDACIÓN ==========
    # Filtrar datos inválidos o atípicos (menos restrictivo que antes)
    if (trip_distance is None or trip_distance <= 0 or trip_distance >= 100 or
        passenger_count is None or passenger_count <= 0 or passenger_count > 6 or
        datetime is None or
        fare_amount is None or fare_amount <= 0 or fare_amount >= 200):
        return None
    
    # ========== EXTRACCIÓN DE FEATURES TEMPORALES ==========
    hour_value = float(datetime.hour)                # 0-23
    day_of_week = float(datetime.weekday() + 1)      # 1=Lunes, 7=Domingo
    
    # ========== ESCALADO PARA MEJOR CONVERGENCIA CON ReLU ==========
    # Z-score aproximado basado en conocimiento del dominio
    
    # Feature 1: Distancia del viaje (media ~3, std ~5)
    trip_distance_scaled = (trip_distance - 3.0) / 5.0
    
    # Feature 2: Número de pasajeros (media ~1.5, std ~1)
    passenger_count_scaled = (passenger_count - 1.5) / 1.0
    
    # Feature 3: Hora del día (media ~12, std ~7)
    hour_scaled = (hour_value - 12.0) / 7.0
    
    # Feature 4: Día de la semana (media ~4, std ~2)
    day_scaled = (day_of_week - 4.0) / 2.0
    
    # Target: Tarifa (media ~15, std ~10) - Sin escalar para salida ReLU
    # IMPORTANTE: La salida ReLU puede predecir valores positivos ilimitados
    fare_target = float(fare_amount)
    
    # ========== RETORNAR FORMATO COMPATIBLE CON ELEPHAS ==========
    features = [
        float(trip_distance_scaled),
        float(passenger_count_scaled),
        float(hour_scaled),
        float(day_scaled)
    ]
    
    label = fare_target  # Target sin escalar para ReLU
    
    return (features, label)

print("Función de feature engineering con escalado definida")

# Aplicar escalado DISTRIBUIDO al RDD
print("Aplicando escalado distribuido.")

rdd_scaled = rdd_features.map(
    lambda row: extract_and_scale_features(row)
).filter(lambda x: x is not None).cache()

# Contar registros válidos
total_scaled = rdd_scaled.count()

print(f"\n Escalado completado")
print(f"  Registros originales: {rdd_features.count():,}")
print(f"  Registros escalados: {total_scaled:,}")
print(f"  Registros filtrados: {rdd_features.count() - total_scaled:,}")

# Mostrar ejemplo de dato escalado
print("\nEjemplo de registro escalado:")
sample = rdd_scaled.take(1)[0]
print(f"  Features (escaladas): {[f'{x:.3f}' for x in sample[0]]}")
print(f"  Label (fare original): ${sample[1]:.2f}")


Función de feature engineering con escalado definida
Aplicando escalado distribuido.

 Escalado completado
  Registros originales: 2,964,624
  Registros escalados: 2,722,784
  Registros filtrados: 241,840

Ejemplo de registro escalado:
  Features (escaladas): ['-0.256', '-0.500', '-1.714', '-1.500']
  Label (fare original): $17.70


In [7]:
#----------------------División Train/Test---------------------------------------
# Dividiendo los datos en entrenamiento (80%) y prueba (20%)
train_rdd, test_rdd = rdd_scaled.randomSplit([0.8, 0.2], seed=42)

# Cachear para mejor rendimiento
train_rdd = train_rdd.cache()
test_rdd = test_rdd.cache()

train_count = train_rdd.count()
test_count = test_rdd.count()

print(f"\n✓ División de datos completada")
print(f"  Train: {train_count:,} registros ({train_count/total_scaled*100:.1f}%)")
print(f"  Test:  {test_count:,} registros ({test_count/total_scaled*100:.1f}%)")

#---------------------------------------------------------------------------------


✓ División de datos completada
  Train: 2,178,428 registros (80.0%)
  Test:  544,356 registros (20.0%)


In [8]:
#----------------------Construcción del Modelo Keras----------------------------
# Arquitectura:
# - Capa 1: 8 neuronas, activación ReLU
# - Capa 2: 4 neuronas, activación ReLU  
# - Capa 3: 1 neurona, activación ReLU

def create_keras_model():
    model = Sequential([
        Dense(8, activation='relu', input_shape=(4,), name='capa_oculta_1'),   # 4 features de entrada
        Dense(4, activation='relu', name='capa_oculta_2'),
        Dense(1, activation='relu', name='capa_salida')  # ReLU en salida para valores positivos
    ])
    
    # Compilar el modelo
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mse',  # Error cuadrático medio para regresión
        metrics=['mae']  # Error absoluto medio como métrica adicional
    )
    
    return model

# Crear modelo local
keras_model = create_keras_model()
print("\n Modelo Keras creado con arquitectura ReLU")
print(keras_model.summary())

#---------------------------------------------------------------------------------


 Modelo Keras creado con arquitectura ReLU


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ capa_oculta_1 (Dense)           │ (None, 8)              │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa_oculta_2 (Dense)           │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa_salida (Dense)             │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 81 (324.00 B)

 Trainable params: 81 (324.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
#----------------------Preparación para Elephas (MÉTODO CORREGIDO)---------------
print("\nPreparando datos para Elephas...")

# Crear el formato específico que espera Elephas
def prepare_for_elephas(row):
    """Prepara cada fila para Elephas"""
    features, label = row
    # Elephas espera: (features_array, label)
    return (features.tolist() if hasattr(features, 'tolist') else features, float(label))

# Aplicar transformación y cachear
train_elephas_rdd = train_rdd.map(prepare_for_elephas).cache()
test_elephas_rdd = test_rdd.map(prepare_for_elephas).cache()

# Forzar evaluación
train_elephas_rdd.count()
test_elephas_rdd.count()

print("✓ RDDs preparados para Elephas")
print(f"  Muestra train: {train_elephas_rdd.take(1)}")

#---------------------------------------------------------------------------------


Preparando datos para Elephas...
✓ RDDs preparados para Elephas
  Muestra train: [([-0.256, -0.5, -1.7142857142857142, -1.5], 17.7)]


In [11]:
#----------------------PASO 10: Crear el SparkModel de Elephas------------------------
print("\n" + "="*80)
print("PASO 10: Configurando Elephas para entrenamiento distribuido")
print("="*80)

# Configuración de Elephas
from elephas.spark_model import SparkModel

# Parámetros de entrenamiento distribuido
BATCH_SIZE = 256        # Tamaño de batch por worker
EPOCHS = 10             # Número de épocas
NUM_WORKERS = 4         # Número de workers (ajusta según tu máquina)
VERBOSE = 1             # Nivel de verbosidad

print(f"\n📋 Configuración de entrenamiento distribuido:")
print(f"  • Batch size: {BATCH_SIZE}")
print(f"  • Épocas: {EPOCHS}")
print(f"  • Workers: {NUM_WORKERS}")
print(f"  • Registros train: {train_count:,}")
print(f"  • Registros test: {test_count:,}")

# Crear el modelo distribuido de Elephas
print("\n🔧 Creando SparkModel de Elephas...")

spark_model = SparkModel(
    model=keras_model,           # Tu modelo Keras
    frequency='epoch',           # Frecuencia de sincronización ('epoch' o 'batch')
    mode='asynchronous',         # Modo asíncrono para mejor rendimiento
    num_workers=NUM_WORKERS      # Número de workers para paralelizar
)

print("✓ SparkModel creado exitosamente")
print(f"  Modo: Asíncrono (mejor rendimiento)")
print(f"  Sincronización: Por época")

#---------------------------------------------------------------------------------


PASO 10: Configurando Elephas para entrenamiento distribuido

📋 Configuración de entrenamiento distribuido:
  • Batch size: 256
  • Épocas: 10
  • Workers: 4
  • Registros train: 2,178,428
  • Registros test: 544,356

🔧 Creando SparkModel de Elephas...
✓ SparkModel creado exitosamente
  Modo: Asíncrono (mejor rendimiento)
  Sincronización: Por época


In [14]:
#----------------------PASO 11: Entrenar el modelo distribuido-------------------
print("\n" + "="*80)
print("PASO 11: Entrenamiento distribuido con Elephas")
print("="*80)

import time

print("\n🎯 Iniciando entrenamiento distribuido...")
print("⏱️  Este proceso puede tomar varios minutos con ~2.2M registros\n")

start_time = time.time()

# ENTRENAR EL MODELO DISTRIBUIDO
# Elephas distribuye automáticamente el entrenamiento entre workers
spark_model.fit(
    train_elephas_rdd,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=VERBOSE,
    validation_split=0.1  # 10% de train para validación
)

training_time = time.time() - start_time

print(f"\n✓ Entrenamiento completado en {training_time/60:.2f} minutos")
print(f"  Tiempo por época: ~{training_time/EPOCHS:.1f} segundos")

#---------------------------------------------------------------------------------


PASO 11: Entrenamiento distribuido con Elephas

🎯 Iniciando entrenamiento distribuido...
⏱️  Este proceso puede tomar varios minutos con ~2.2M registros

>>> Fit model


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 24.0 failed 1 times, most recent failure: Lost task 1.0 in stage 24.0 (TID 123) (DESKTOP-OEGMP3C executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed). Consider setting 'spark.sql.execution.pyspark.udf.faulthandler.enabled' or'spark.python.worker.faulthandler.enabled' configuration to 'true' for the better Python traceback.
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:678)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:663)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:35)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:1034)
	at org.apache.spark.api.python.PythonRunner$$anon$3